In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
import matplotlib.pyplot as plt

Using TensorFlow backend.


pygame 2.0.0 (SDL 2.0.12, python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()
sound = mixer.Sound('alarm.wav')

try:
    face = cv2.CascadeClassifier('haar-cascade-files/haarcascade_frontalface_alt.xml')
    leye = cv2.CascadeClassifier('haar-cascade-files/haarcascade_lefteye_2splits.xml')
    reye = cv2.CascadeClassifier('haar-cascade-files/haarcascade_righteye_2splits.xml')
#     eyePair_cascade = cv2.CascadeClassifier('./haar-cascade/haarcascades_haarcascade_mcs_eyepair_big.xml') #eye_pair
    print("Cascade loaded successfully")
except:
    print("Can not open file")


lbl=['Close','Open']
try:
    model = load_model('my_modelexp1.h5')
    print("Model loaded successfully")
except:
    print("can not load model")
    
path = os.getcwd()

font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
right_eye_pred=[99]
left_eye_pred=[99]

Cascade loaded successfully
Model loaded successfully


In [4]:
try:
    cap = cv2.VideoCapture(0)
except:
    print("Can not initialize camera")

while(True):
    try:
        ret, img = cap.read()
    except:
        print("Can not read image")
        
    height,width = img.shape[:2] 

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(5,5))
    
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)
    
    defaultPicture = np.zeros((60,150,1)) # black picture
    cv2.rectangle(img, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        color = img[y:y+h, x:x+w]
        cv2.rectangle(img, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

        for (x,y,w,h) in right_eye:
            cv2.rectangle(color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            r_eye=img[y:y+h,x:x+w]
            count=count+1
            r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
            r_eye = cv2.resize(r_eye,(24,24))
            r_eye= r_eye/255
            r_eye=  r_eye.reshape(24,24,-1)
            r_eye = np.expand_dims(r_eye,axis=0)
            right_eye_pred = model.predict_classes(r_eye)
            if(right_eye_pred[0]==0):
                lbl='Open' 
            if(right_eye_pred[0]==1):
                lbl='Closed'
            break

        for (x,y,w,h) in left_eye:
            cv2.rectangle(color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2)
            l_eye=img[y:y+h,x:x+w]
            count=count+1
            l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
            l_eye = cv2.resize(l_eye,(24,24))
            l_eye= l_eye/255
            l_eye=l_eye.reshape(24,24,-1)
            l_eye = np.expand_dims(l_eye,axis=0)
            left_eye_pred = model.predict_classes(l_eye)
            if(left_eye_pred[0]==0):
                lbl='Open'   
            if(left_eye_pred[0]==1):
                lbl='Closed'
            break

    if(right_eye_pred[0]==0 and left_eye_pred[0]==0):
        score=score+1
        cv2.putText(img,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(img,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(img,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),img)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(img,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('IMAGE',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


NameError: name 'ex' is not defined

In [ ]:
print(img)

In [ ]:
cap.read()

In [ ]:
plt.imshow(l_eye)